<a href="https://colab.research.google.com/github/hiydavid/numerai-models/blob/main/numerai_model_selector_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numerai Model Selection * WIP *
---

Selec Pickled Model
* SKLearn OLS base model
* SKLearn Random Forest model
* Tensorflow Decision Forest model


---
# Load data & libraries

In [1]:
# set drive location
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)
path = '/content/drive/MyDrive/Colab/numerai/saved_models/'

Mounted at /content/drive/


In [2]:
# install dependencies
!pip install -Uqq numerapi==2.4.5 # tensorflow==2.5.1 tensorflow_decision_forests

In [3]:
# import dependencies
import pandas as pd
import numerapi
import pickle

In [4]:
# download the latest tournament dataset (takes around 30s)
tournament_data_url = "https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz"
tournament_data = pd.read_csv(tournament_data_url)

In [5]:
# find only the feature columns
feature_cols = tournament_data.columns[tournament_data.columns.str.startswith('feature')]

In [6]:
# select those columns out of the training dataset
live_features = tournament_data[feature_cols]

--- 
# Convert dataframes to  (TFDF only)

In [7]:
# # find only the feature columns
# feature_cols = tournament_data.columns[
#     tournament_data.columns.str.startswith('feature')
#     ].append(pd.Index(['target']))

In [8]:
# # convert features data to ds object
# live_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
#     tournament_data[feature_cols], 
#     label="target",
#     task=tfdf.keras.Task.REGRESSION
# )

---
# Load model performances

---
# Load model based on best performance

In [9]:
# load base model
with open(path + "sklearn_base_model.pkl", 'rb') as file:
    model = pickle.load(file)

---
# Predict

In [10]:
# predict the target on the live features
predictions = model.predict(live_features)

In [11]:
# predictions must have an `id` column and a `prediction_kazutsugi` column
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions
predictions_df.head()

,id,prediction_kazutsugi
0,n0003aa52cab36c2,0.485069
1,n000920ed083903f,0.489169
2,n0038e640522c4a6,0.529772
3,n004ac94a87dc54b,0.495391
4,n0052fe97ea0c05f,0.499209


--- 
# Submit

In [12]:
# Get your API keys and model_id from https://numer.ai/submit
public_id = "WPZQRGGZADFULXQ3RRMR6OJQWALLSQTM"
secret_key = "T5ZJLYRT4652F244OLVRTITXH5W4TJRQ553RLJZLQTX7EPRU647WU33ITZPHP3FU"
model_id = "4d9bcd2c-69cb-4e17-a3f3-a4b2530cbcb8"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [13]:
# Upload your predictions
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=model_id)

2021-09-11 20:40:51,459 INFO numerapi.base_api: uploading predictions...
